# Speculative Decoding - Simplified

This notebook demonstrates the speculative decoding workflow .

## 1. Setup Workspace

In [ ]:
from rl_spec_dec_utils import setup_workspace, run_rl_training_pipeline, run_draft_model_pipeline, prepare_combined_model_for_deployment, deploy_speculative_decoding_endpoint, test_deployment

# Setup Azure ML workspace and registry connections
ml_client, registry_ml_client = setup_workspace(registry_name="test_centralus")

Found the config file in: config.json
c:\Users\ynagaraj\AppData\Local\anaconda3\envs\p310\lib\site-packages\mlflow\__init__.py:41: UserWarning: Versions of mlflow (3.1.4) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


✓ Connected to registry: test_centralus
✓ Connected to workspace: rtanase
✓ Resource group: rtanase


: 

## 2. Create Draft Model for Speculative Decoding

In [ ]:
# Train EAGLE3 draft model for speculative decoding
draft_job, draft_status = run_draft_model_pipeline(
    ml_client=ml_client,
    registry_ml_client=registry_ml_client,
    compute_cluster="h100-dedicated",
    num_epochs=1,
    monitor=False  # Set to True to wait for completion
)

## 3. Prepare Combined Model for Deployment

In [ ]:
# Download draft model, download base model, combine and register for deployment
combined_model = prepare_combined_model_for_deployment(
    ml_client=ml_client,
    draft_job_name=draft_job.name,
    base_model_hf_id="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    model_name="grpo-speculative-decoding",
)

## 4. Deploy Speculative Decoding Endpoint

In [ ]:
# Deploy managed online endpoint with speculative decoding
endpoint_name = deploy_speculative_decoding_endpoint(
    ml_client=ml_client,
    combined_model=combined_model,
    instance_type="Standard_NC24ads_A100_v4"
)